# 1: Load Saved Correlation Matrices

In [1]:
import numpy as np
import os

# Define directory containing saved correlation matrices
corr_dir = os.path.expanduser("~/correlation_matrices")

# Map each correlation type to its saved file
correlation_types = {
    "correlation": "correlation_correlation_matrices.npz",
    "partial correlation": "partial correlation_correlation_matrices.npz",
    "tangent": "tangent_correlation_matrices.npz"
}

# Load the matrices
'''
Loop iterates through each item in the dictionary and loads the corresponding npz files
The npz files contain a key called matrices (actual data) 
We print the shape of each matrix to ensure that the correct data is loaded
'''

correlation_matrices = {}
for kind, filename in correlation_types.items():
    path = os.path.join(corr_dir, filename)
    data = np.load(path)
    correlation_matrices[kind] = data["matrices"]
    print(f"{kind}: {correlation_matrices[kind].shape}")


correlation: (155, 39, 39)
partial correlation: (155, 39, 39)
tangent: (155, 39, 39)


1. 155 - number of subjects
2. 39 x 39 - pairwise relationships between nodes (fully connected graph)

# 2: Load Labels

In [2]:
import numpy as np
import os

# Define path to the labels file 
label_path = os.path.expanduser("~/time_series_data/subject_labels.npy")
labels = np.load(label_path)

# Confirm it's correct
print("Labels shape:", labels.shape)
print("Unique labels and counts:", np.unique(labels, return_counts=True))


Labels shape: (155,)
Unique labels and counts: (array([0, 1]), array([122,  33]))


1. 155 subjects 
2. Child(0) - 122, Adult(1) - 33

# 3: Define Thresholds and Node Feature Strategies

In [3]:
# Define thresholds and correlation types
threshold_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
correlation_kinds = list(correlation_matrices.keys())  # keys from the dictionary (3)

# Node feature strategies
# We'll define functions for each strategy
from sklearn.decomposition import PCA

# PCA feature extraction
def get_pca_features(corr_matrix, n_components=10):
    abs_mat = np.abs(corr_matrix)
    return PCA(n_components=n_components).fit_transform(abs_mat)

# Node strength feature extraction
def get_node_strength_features(corr_matrix):
    return np.sum(np.abs(corr_matrix), axis=1).reshape(-1, 1)  # shape: (n_nodes, 1)

# Mapping of strategy name to function
# Map each feature extraction strategy to its corresponding function 
feature_strategies = {
    "pca": get_pca_features,
    "strength": get_node_strength_features
}


1. def get_pca_features(corr_matrix, n_components=10):
- Function to get PCA features 
- This function takes a correlation matrix, computes its absolute values, and applies PCA to reduce the number of components (nodes' features)
- n_components=10 means that only the first 10 principal components are kept

2. def get_node_strength_features(corr_matrix):
- Function to get node strength features
- This function computes the node strength by summing the absolute values of the correlations for each node
- The result is reshaped to ensure it's in the form (n_nodes, 1) - one value per node 
- The np.sum function is applied across rows (axis=1), which means we're summing the absolute correlations for each node. The result is a 1D array of shape (n_nodes,). However, pytorch expects a 2D format (n_nodes, 1)

# 4: Generate PyTorch Geometric Graphs

1. Thresholding function:
- takes a correlation matrix and a threshold value as input
- creates and adjacency matrix where the abs value of the correlation must be greater that the threshold (np.abs(corr_matrix) > threshold):
   - 1 represents an edge (correlation above the threshold)
   - 0 represents no edge (correlation below the threshold)
- The line np.fill_diagonal(adj, 0) removes self-connections by setting the diagonal of the matrix to 0.
- The np.nonzero(adj) function returns the indices of the non-zero entries in the adjacency matrix (i.e., the edges in the graph).
- The edge_index is returned as a tensor to be used in PyTorch Geometric.

2. Graph Generation

- graphh_dict stores the generated graphs. The key is a tuple of (correlation_type, threshold, feature_type), and the value is a list of Data objects from PyTorch Geometric.

- For each correlation type:
  - matrices = correlation_matrices[corr_type]: This fetches the correlation matrix based on the correlation type (correlation, partial correlation, or tangent).
- For each threshold:
  - The code loops through threshold_values and generates graphs using the corresponding correlation matrix (mat).
- Feature Extraction:
  - The node features are extracted using the corresponding feature function (feat_func) from the feature_strategies dictionary.

These features are then converted into a PyTorch tensor (x).

- The generated graph is stored as a Data object in PyTorch Geometric format:
   1. x: Node features (after applying the feature extraction strategy).
   2. edge_index: The edges, determined by thresholding the correlation matrix.
   3. y: The label of the subject.
- The graph is appended to the list of graphs for the (corr_type, threshold, feat_type) combination, which is stored in graph_dict.

In [4]:
import torch
from torch_geometric.data import Data

'''
Function applies a threshold to the correlation matrix to generate a 
binary adjacency matrix representing the edges of the graph.
'''

def threshold_edges(corr_matrix, threshold):
    # Create binary adjacency mask where abs(corr) > threshold (excluding self-connections)
    adj = (np.abs(corr_matrix) > threshold).astype(int) #1=edge, above threshold vice versa
    np.fill_diagonal(adj, 0) # remove self connections by setting the diagonal of the matrix to 0
    edge_index = np.array(np.nonzero(adj)) # return indices of the non zero entries in the adj matrix (i.e edges in the graph)
    return torch.tensor(edge_index, dtype=torch.long)

# Main graph generation
'''
build graphs for each combination of correlation type, threshold, and feature extraction strategy
uses the thresholded correlation matrices as the edges, computes node features using PCA or node strength, and stores the resulting graphs in a dictionary.
'''
#stores generated graphs
graph_dict = {}  # key = (corr_type, threshold, feature_type), value = list of Data objects

for corr_type in correlation_kinds:
    matrices = correlation_matrices[corr_type] #matrices corresponding to the correlation type loaded
    for threshold in threshold_values:
        for feat_type, feat_func in feature_strategies.items():
            graphs = []
            for i in range(len(matrices)):
                mat = matrices[i]
                x = torch.tensor(feat_func(mat), dtype=torch.float)  # node features
                edge_index = threshold_edges(mat, threshold)          # edges
                y = torch.tensor([labels[i]], dtype=torch.long)       # label
                graphs.append(Data(x=x, edge_index=edge_index, y=y))
            graph_dict[(corr_type, threshold, feat_type)] = graphs
            print(f"✓ Done: {corr_type}, threshold={threshold}, features={feat_type}")


✓ Done: correlation, threshold=0.1, features=pca
✓ Done: correlation, threshold=0.1, features=strength
✓ Done: correlation, threshold=0.2, features=pca
✓ Done: correlation, threshold=0.2, features=strength
✓ Done: correlation, threshold=0.3, features=pca
✓ Done: correlation, threshold=0.3, features=strength
✓ Done: correlation, threshold=0.4, features=pca
✓ Done: correlation, threshold=0.4, features=strength
✓ Done: correlation, threshold=0.5, features=pca
✓ Done: correlation, threshold=0.5, features=strength
✓ Done: correlation, threshold=0.6, features=pca
✓ Done: correlation, threshold=0.6, features=strength
✓ Done: correlation, threshold=0.7, features=pca
✓ Done: correlation, threshold=0.7, features=strength
✓ Done: correlation, threshold=0.8, features=pca
✓ Done: correlation, threshold=0.8, features=strength
✓ Done: correlation, threshold=0.9, features=pca
✓ Done: correlation, threshold=0.9, features=strength
✓ Done: partial correlation, threshold=0.1, features=pca
✓ Done: partial 

# 5: Define Train/Test Split and Class Weights

In [14]:
from sklearn.model_selection import train_test_split

def prepare_data(graphs, labels, test_size=0.2, random_state=42):
    y = np.array([g.y.item() for g in graphs]) #create numpy array y that contains the labels for each graph
    train_idx, test_idx = train_test_split( #split graphs to training and test sets
        np.arange(len(graphs)), #generates an array of indices for the graphs
        test_size=test_size, #defines the ratio of data to be used for the test set 
        stratify=y, # ensures same % of children and adults used as in the original dataset
        random_state=random_state #ensure that split is same every time 
    )

    train_graphs = [graphs[i] for i in train_idx] #creates training set by selecting graphs corresponding to the indices of train_idx
    test_graphs = [graphs[i] for i in test_idx] #create test set similarly  

    # Compute class weights
    class_counts = torch.bincount(torch.tensor(y[train_idx])) #count of samples for each class in training set
    num_samples = len(train_idx) #total num of samples in the training set 
    num_classes = 2 # num of classes in the classification problem
    class_weights = num_samples / (num_classes * class_counts.float()) #computes class weights based on inverse of class freq
    #underrepresented classes get higher weights

    # Print out class weights
    print("Class Weights:", class_weights)

    return train_graphs, test_graphs, class_weights


1 Function Definition:

- prepare_data is defined to take in graphs, labels, a test size (default 0.2, meaning 20% test data), and a random seed for reproducibility.

- It creates a numpy array y that contains the labels for each graph (g.y.item() fetches the label for graph g).

2. train_test_split:

- The train_test_split function splits the data (graphs) into training and test sets:

  - np.arange(len(graphs)) generates an array of indices for the graphs.

  - test_size=test_size defines the ratio of data to be used for the test set.

  - stratify=y ensures that the train and test sets will have a proportional distribution of labels (i.e., the same percentage of children and adults as in the original dataset).

  - random_state=random_state ensures that the split is the same every time the code is run (for reproducibility).

# 6: Define the GCN Model and Training Loop

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

# Define a simple 2-layer GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim=32): #input has input_dim features, output has hidden_dim features
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, 2)  # Binary classification

    def forward(self, data):
        # data.x = node features(inputs)
        # data.edge_index = edges (graph connectivity)
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index) #passes the input node features x through first gcn layer
        x = F.relu(x) #applies ReLU activation function to introduce non-linearity
        x = self.conv2(x, edge_index) #passez output from 1st layer to the 2nd layer
        x = F.relu(x) #applies ReLU again
        x = torch.mean(x, dim=0)  # global mean pooling over nodes(avg of node features across entire graph)
        return self.fc(x) #pooled output passed through the fully connected layer for classification


- input_dim: dimension of input features (number of features per node).
- hidden_dim: dimension of hidden layers (32 by default).
- self.conv1: The first graph convolution layer that transforms the input node features into hidden features.
- self.conv2: takes the hidden_dim features and outputs hidden_dim features 
- self.fc: a fully connected layer that takes hidden_dim features and outputs 2 values (binary classification)

#  7: Training and Evaluation Functions

In [7]:
#model training func for each epoch 
# model = GCN to be trained 
#loader = dataloader that provides the training data in batches
#optimizer = updates the model parameters
#loss_fn = loss function used to compute error 
def train(model, loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0 #counter for loss
    correct = 0 #counter for correct classifications
    for batch in loader: #iterate over each batch of data in the training loader
        batch = batch.to(device) #move data to device
        optimizer.zero_grad() #clear accumulated gradients from previous iterations
        out = model(batch) # performs a forward pass through the model
        loss = loss_fn(out.unsqueeze(0), batch.y) #predicted output compared to true labels batch.y
        loss.backward() #computes gradients of loss with respect to model's parameters
        optimizer.step() #update the model parameters based on the gradients
        total_loss += loss.item() #accumulate total loss
        pred = out.argmax(dim=0) #finds index of max predicted val
        correct += int(pred == batch.y.item()) #checks if prediction matches true label and updates accuracy
    avg_loss = total_loss / len(loader) #calculate avg loss over all batches
    accuracy = correct / len(loader) #computes training accuracy
    print(f"Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.4f}")
    return avg_loss, accuracy #avg loss + accuracy

#evaluate on test set
# model = trained model to evaluate 
# loader = data loader for the test/validation set
def evaluate(model, loader, device):
    model.eval()
    correct = 0 #counter for correct classified samples
    with torch.no_grad(): #ensures no gradients calculated during evaluation
        for batch in loader: #iterate over the batches in test set
            batch = batch.to(device)
            out = model(batch) #forward pass to get predictions of current batch
            pred = out.argmax(dim=0) #finds class with highest predicted probability using argmax
            correct += int(pred == batch.y.item()) #compares predicted class to true class and increments correct if equal
    return correct / len(loader)


# 8: Train and Evaluate on One Configuration


In [8]:
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pick one configuration
config = ("tangent", 0.5, "pca")
graphs = graph_dict[config]

# Prepare train/test split and class weights
train_graphs, test_graphs, class_weights = prepare_data(graphs, labels)

# Dataloaders
train_loader = DataLoader(train_graphs, batch_size=1, shuffle=True)
test_loader = DataLoader(test_graphs, batch_size=1)

# Define model, optimizer, loss
input_dim = train_graphs[0].x.shape[1]
model = GCN(input_dim=input_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

# Training loop
epochs = 20
for epoch in range(1, epochs + 1):
    print(f"Epoch {epoch}")
    train_loss, train_acc = train(model, train_loader, optimizer, loss_fn, device)
    test_acc = evaluate(model, test_loader, device)
    print(f"Test Accuracy: {test_acc:.4f}")
    print("------")


tensor([0.6327, 2.3846])
Epoch 1
Train Loss: 0.6823, Train Accuracy: 0.7823
Test Accuracy: 0.7742
------
Epoch 2
Train Loss: 0.5286, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 3
Train Loss: 0.5410, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 4
Train Loss: 0.5424, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 5
Train Loss: 0.5254, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 6
Train Loss: 0.5349, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 7
Train Loss: 0.5334, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 8
Train Loss: 0.5226, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 9
Train Loss: 0.5245, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 10
Train Loss: 0.5167, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 11
Train Loss: 0.5192, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 12
Train Loss: 0.5233, Train Accuracy: 0.7903
Test Accuracy: 0.7742
------
Epoch 13
Tra

# 9: Loop Over All Correlation × Threshold × Feature Combos

In [9]:
import pandas as pd

results = [] #store final evaluation results

#iterate over all combinations of correlation type, threshold, and feature type 
for corr_type in correlation_kinds:
    for threshold in threshold_values:
        for feat_type in feature_strategies:
            print(f"\n▶️ Running: {corr_type}, threshold={threshold}, features={feat_type}")
            graphs = graph_dict[(corr_type, threshold, feat_type)] #retreive corresponding graph from graph_dict based on the combination
            #split into training and test sets, calculate class weights, create data loaders 
            train_graphs, test_graphs, class_weights = prepare_data(graphs, labels) 
            train_loader = DataLoader(train_graphs, batch_size=1, shuffle=True)
            test_loader = DataLoader(test_graphs, batch_size=1)
            
            input_dim = train_graphs[0].x.shape[1] #input dim set based on num of features in 1st training graph
            model = GCN(input_dim=input_dim).to(device) 
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01) #adam optimizer
            loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device)) #crossentropyloss

            best_test_acc = 0 #test accuracy counter
            #training loop for each configuration
            for epoch in range(1, 11):  # model trained for 10 epochs
                train(model, train_loader, optimizer, loss_fn, device)
                test_acc = evaluate(model, test_loader, device)
                if test_acc > best_test_acc:
                    best_test_acc = test_acc #best test accuracy updated if higher accuracy found
            
            #after training + evaluation, results are added to results list
            results.append({
                "Correlation": corr_type,
                "Threshold": threshold,
                "Features": feat_type,
                "BestTestAcc": round(best_test_acc, 4)
            })



▶️ Running: correlation, threshold=0.1, features=pca
tensor([0.6327, 2.3846])
Train Loss: 0.6620, Train Accuracy: 0.7903
Train Loss: 0.5115, Train Accuracy: 0.7903
Train Loss: 0.5503, Train Accuracy: 0.7903
Train Loss: 0.5311, Train Accuracy: 0.7903
Train Loss: 0.5280, Train Accuracy: 0.7903
Train Loss: 0.5250, Train Accuracy: 0.7903
Train Loss: 0.5176, Train Accuracy: 0.7903
Train Loss: 0.5243, Train Accuracy: 0.7903
Train Loss: 0.5176, Train Accuracy: 0.7903
Train Loss: 0.5194, Train Accuracy: 0.7903

▶️ Running: correlation, threshold=0.1, features=strength
tensor([0.6327, 2.3846])
Train Loss: 0.5337, Train Accuracy: 0.7823
Train Loss: 0.5317, Train Accuracy: 0.7903
Train Loss: 0.5616, Train Accuracy: 0.7903
Train Loss: 0.5364, Train Accuracy: 0.7903
Train Loss: 0.5615, Train Accuracy: 0.7903
Train Loss: 0.5243, Train Accuracy: 0.7903
Train Loss: 0.5173, Train Accuracy: 0.7903
Train Loss: 0.5205, Train Accuracy: 0.7903
Train Loss: 0.5270, Train Accuracy: 0.7903
Train Loss: 0.5263, 

In [10]:

#results_df_sorted = pd.DataFrame(results).sort_values("BestTestAcc", ascending=False).reset_index(drop=True)

# Display test accuracy results
#print(results_df_sorted)


- results: Contains the list of all results from previous training runs, which includes the best test accuracy for each configuration.

- sort_values("BestTestAcc", ascending=False): This sorts the results based on the BestTestAcc column in descending order (higher accuracy first).

- reset_index(drop=True): This resets the index of the DataFrame after sorting, so the index is sequential starting from 0.

In [11]:
results_df_sorted = pd.DataFrame(results).sort_values("BestTestAcc", ascending=False).reset_index(drop=True)

# Group by Correlation first, then Threshold, then Features
grouped_results_df = results_df_sorted.groupby(['Correlation', 'Threshold', 'Features']).agg({
    'BestTestAcc': 'mean'
}).reset_index()

# Display the grouped results
print(grouped_results_df)


            Correlation  Threshold  Features  BestTestAcc
0           correlation        0.1       pca       0.7742
1           correlation        0.1  strength       0.7742
2           correlation        0.2       pca       0.7742
3           correlation        0.2  strength       0.7742
4           correlation        0.3       pca       0.7742
5           correlation        0.3  strength       0.7742
6           correlation        0.4       pca       0.7742
7           correlation        0.4  strength       0.7742
8           correlation        0.5       pca       0.7742
9           correlation        0.5  strength       0.7742
10          correlation        0.6       pca       0.7742
11          correlation        0.6  strength       0.7742
12          correlation        0.7       pca       0.7742
13          correlation        0.7  strength       0.7742
14          correlation        0.8       pca       0.7742
15          correlation        0.8  strength       0.7742
16          co

# 10: Modifying the GCN Model

- input_dim: The number of features in the input nodes.
- hidden_dim: The number of neurons in each hidden layer (64).
- num_layers: The number of layers in the GCN (4).
- output_dim: The number of output classes (2).

In [12]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class MultiLayerGCN(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=4, output_dim=2):
        super(MultiLayerGCN, self).__init__()
        self.num_layers = num_layers

        # Create a list of GCN layers
        self.layers = nn.ModuleList() # A list to hold the different GCN layers.
        self.layers.append(GCNConv(input_dim, hidden_dim))  # First layer(maps input features to hidden layer)

        # Add hidden layers
        for _ in range(num_layers - 2): 
            self.layers.append(GCNConv(hidden_dim, hidden_dim)) #second layer 

        # Output layer (mapping to number of classes)
        self.layers.append(GCNConv(hidden_dim, output_dim))

        # Optional: a fully connected layer for further non-linearity (if needed)
        # self.fc = nn.Linear(output_dim, output_dim)

    def forward(self, data): #defines the forward pass
        # data.x = node features(inputs)
        # data.edge_index = edges (graph connectivity)
        x, edge_index = data.x, data.edge_index 

        # Apply each GCN layer with ReLU activation
        for i in range(self.num_layers - 1): #loop through all layers except last one
            x = self.layers[i](x, edge_index)
            x = F.relu(x) #relu activation for ach GCN layer 

        # Apply final GCN layer (without activation)
        x = self.layers[-1](x, edge_index)

        # Optional: Apply a fully connected layer if used
        # x = self.fc(x)
        
        # Return final graph-level embedding
        return x.mean(dim=0)  #global mean pooling over nodes(avg of node features across entire graph)


Summary:
- The model defines multiple layers of graph convolutions (GCNConv), where the first layers are hidden layers and the last layer produces output to map to the number of classes.
- ReLU activation is used after each hidden layer to introduce non-linearity.
- Global mean pooling is applied to aggregate node embeddings into a graph-level embedding, which is suitable for graph-level classification tasks.

In [13]:
import pandas as pd

results = []

for corr_type in correlation_kinds:
    for threshold in threshold_values:
        for feat_type, feat_func in feature_strategies.items():
            print(f"\n▶️ Running: {corr_type}, threshold={threshold}, features={feat_type}")
            
            # Get the graphs for the current combination
            graphs = graph_dict[(corr_type, threshold, feat_type)]
            train_graphs, test_graphs, class_weights = prepare_data(graphs, labels)
            
            # Define DataLoader for train and test sets
            train_loader = DataLoader(train_graphs, batch_size=1, shuffle=True)
            test_loader = DataLoader(test_graphs, batch_size=1)

            # Define the model, optimizer, and loss function
            input_dim = train_graphs[0].x.shape[1]
            model = MultiLayerGCN(input_dim=input_dim, hidden_dim=64, num_layers=4).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
            loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device)) 

            best_test_acc = 0
            for epoch in range(1, 11):  # shorter training per combo
                train_loss, train_acc = train(model, train_loader, optimizer, loss_fn, device)
                test_acc = evaluate(model, test_loader, device)
                
                if test_acc > best_test_acc:
                    best_test_acc = test_acc

            # Append the results for each configuration
            results.append({
                "Correlation": corr_type,
                "Threshold": threshold,
                "Features": feat_type,
                "BestTestAcc": round(best_test_acc, 4)
            })
# Convert results to a DataFrame for easier analysis
results_df = pd.DataFrame(results)

# Sort results based on best test accuracy
results_df_sorted = results_df.sort_values("BestTestAcc", ascending=False).reset_index(drop=True)
print(results_df_sorted)



▶️ Running: correlation, threshold=0.1, features=pca
tensor([0.6327, 2.3846])
Train Loss: 0.5799, Train Accuracy: 0.7661
Train Loss: 0.5551, Train Accuracy: 0.7903
Train Loss: 0.5401, Train Accuracy: 0.7903
Train Loss: 0.5399, Train Accuracy: 0.7903
Train Loss: 0.5222, Train Accuracy: 0.7903
Train Loss: 0.5190, Train Accuracy: 0.7903
Train Loss: 0.5305, Train Accuracy: 0.7903
Train Loss: 0.5281, Train Accuracy: 0.7903
Train Loss: 0.5218, Train Accuracy: 0.7903
Train Loss: 0.5197, Train Accuracy: 0.7903

▶️ Running: correlation, threshold=0.1, features=strength
tensor([0.6327, 2.3846])
Train Loss: 0.8168, Train Accuracy: 0.7258
Train Loss: 0.5691, Train Accuracy: 0.7903
Train Loss: 0.5364, Train Accuracy: 0.7903
Train Loss: 0.5195, Train Accuracy: 0.7903
Train Loss: 0.5205, Train Accuracy: 0.7903
Train Loss: 0.5267, Train Accuracy: 0.7903
Train Loss: 0.5189, Train Accuracy: 0.7903
Train Loss: 0.5246, Train Accuracy: 0.7903
Train Loss: 0.5234, Train Accuracy: 0.7903
Train Loss: 0.5229, 

KeyboardInterrupt: 